In [ ]:
import numpy as np 
import pandas as pd 
import os


In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import pandas as pd
import pandas_profiling 

import seaborn as sns
import matplotlib.style as style
import matplotlib.pyplot as plt


from scipy.stats import boxcox_normmax
from scipy.special import boxcox1p
from scipy.stats import norm, skew
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import LocalOutlierFactor

import warnings
warnings.filterwarnings('ignore')

In [ ]:
house_train= pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
house_test= pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')
print("Dataset shape:",'house_train', house_train.shape, 'house_test', house_test.shape)

In [ ]:
k = 10
cols = house_train.corr().nlargest(k, 'SalePrice')['SalePrice'].index
k_corr_matrix = house_train[cols].corr()
plt.figure(figsize=(12, 8))
sns.heatmap(k_corr_matrix, annot=True, cmap=plt.cm.RdBu_r)

In [ ]:
sns.set()
cols = ['SalePrice', 'OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 'FullBath', 'YearBuilt']
sns.pairplot(house_train[cols], size = 2)
plt.show()

In [ ]:
def detect_outliers(x, y, top=5, plot=True):
    lof = LocalOutlierFactor(n_neighbors=40, contamination=0.1)
    x_ =np.array(x).reshape(-1,1)
    preds = lof.fit_predict(x_)
    lof_scr = lof.negative_outlier_factor_
    out_idx = pd.Series(lof_scr).sort_values()[:top].index
    if plot:
        f, ax = plt.subplots(figsize=(9, 6))
        plt.scatter(x=x, y=y, c=np.exp(lof_scr), cmap='RdBu')
    return out_idx

In [ ]:
outs = detect_outliers(house_train['GrLivArea'], house_train['SalePrice'],top=5) 
outs

In [ ]:
from collections import Counter
all_outliers=[]

for feature in quantitative:
    try:
        outs = detect_outliers(house_train[feature], house_train['SalePrice'],top=5, plot=False)
    except:
        continue
    all_outliers.extend(outs)

print(Counter(all_outliers).most_common())

outliers = [30, 88, 462, 523, 632, 1298, 1324] #
for i in outliers:
    if i in all_outliers:
        print(i)

In [ ]:
house_train = house_train.drop(house_train.index[outliers])
house_train.shape

In [ ]:
house_train.reset_index(drop=True, inplace=True)
y_train = house_train['SalePrice']
X_train = house_train.drop(['SalePrice'], axis=1)
X_test = house_test

print( X_train.shape,y_train.shape,X_test.shape)

In [ ]:
numeric_dtypes = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numerics_train = []; numerics_test = []
for i in house_train.columns: 
    if house_train[i].dtype in numeric_dtypes:
        numerics_train.append(i)
house_train.update(house_train[numerics_train].fillna(0)) 

for i in house_test.columns:
    if house_test[i].dtype in numeric_dtypes:
        numerics_test.append(i)
house_test.update(house_test[numerics_test].fillna(0))


In [ ]:
data = pd.concat([X_train, house_test], axis=0, sort=False)
data.drop(['Id'], axis=1, inplace=True)
data.shape

In [ ]:
all_data['MSSubClass'] = all_data['MSSubClass'].apply(str)
all_data['YrSold'] = all_data['YrSold'].astype(str)
all_data['MoSold'] = all_data['MoSold'].astype(str)
na_total = all_data.isnull().sum().sort_values(ascending=False)
na_ratio = (all_data.isnull().sum() / all_data.shape[0]).sort_values(ascending=False)
missing_data = pd.concat([na_total, na_ratio], axis=1, keys=['Total', 'Ratio'])
missing_data.head(10)

In [ ]:
all_data.drop(['PoolQC', 'Utilities', 'Street', 'MiscFeature', ], axis=1, inplace=True)

In [ ]:
X_train['has2ndfloor'] = X_train['2ndFlrSF'].apply(lambda x: 1 if x > 0 else 0)
X_train['hasgarage'] = X_train['GarageArea'].apply(lambda x: 1 if x > 0 else 0)
X_train['hasbsmt'] = X_train['TotalBsmtSF'].apply(lambda x: 1 if x > 0 else 0)
X_train['hasfireplace'] = X_train['Fireplaces'].apply(lambda x: 1 if x > 0 else 0)

X_test['has2ndfloor'] = X_test['2ndFlrSF'].apply(lambda x: 1 if x > 0 else 0)
X_test['hasgarage'] = X_test['GarageArea'].apply(lambda x: 1 if x > 0 else 0)
X_test['hasbsmt'] = X_test['TotalBsmtSF'].apply(lambda x: 1 if x > 0 else 0)
X_test['hasfireplace'] = X_test['Fireplaces'].apply(lambda x: 1 if x > 0 else 0)

In [ ]:
X_test.isnull().sum().sort_values(ascending=False)

In [ ]:
jeevan = []
len_X_train =len(X_train)

for i in X_train.columns:
    counts = X_train[i].value_counts()
    zeros = counts.iloc[0]
    if zeros / len_X_train * 100 > 99.94 :
        jeevan.append(i)

jeevan = list(jeevan)
X_train = np.array(X_train.drop(overfit, axis=1).copy())
y_train = np.array(y_train)
X_test = np.array(X_test.drop(overfit, axis=1).copy())

In [ ]:
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import make_pipeline

from sklearn.linear_model import ElasticNet, Lasso, Ridge, ElasticNetCV, LassoCV, RidgeCV
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.svm import SVR

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

from mlxtend.regressor import StackingCVRegressor

In [ ]:

ridge = make_pipeline(RobustScaler(), RidgeCV(alphas=alphas_alt, cv=kfolds))


lasso = make_pipeline(
    RobustScaler(),
    LassoCV(max_iter=1e7, alphas=alphas2, random_state=42, cv=kfolds))


elasticnet = make_pipeline(
    RobustScaler(),
    ElasticNetCV(max_iter=1e7, alphas=e_alphas, cv=kfolds, l1_ratio=e_l1ratio))

svr = make_pipeline(RobustScaler(), SVR(
    C=20,
    epsilon=0.009,
    gamma=0.0003,
))
gbr = GradientBoostingRegressor(n_estimators=3000,
                                learning_rate=0.05,
                                max_depth=4,
                                max_features='sqrt',
                                min_samples_leaf=15,
                                min_samples_split=10,
                                loss='huber',
                                random_state=42)

lightgbm = LGBMRegressor(
    objective='regression',
    num_leaves=4,
    learning_rate=0.01,
    n_estimators=5000,
    max_bin=200
)

xgboost = XGBRegressor(learning_rate=0.01,
                       n_estimators=3460,
                       max_depth=5,
                       min_child_weight=0,
                       gamma=0,
                       subsample=0.7,
                       colsample_bytree=0.7,
                       objective='reg:squarederror',
                       nthread=-1,
                       scale_pos_weight=1,
                       seed=27,
                       reg_alpha=0.00006)

In [ ]:
dhoni_model = StackingCVRegressor(regressors=(ridge, lasso, elasticnet, gbr, xgboost, lightgbm),
                                meta_regressor=xgboost,
                                use_features_in_secondary=True)

In [ ]:

dhoni_model.fit(X_train, y_train) 


In [ ]:

result = dhoni_model.predict(X_test)